In [ ]:
!pip install langchain
# Install the new package if not already
%pip install -U langchain-openai
from langchain_openai import ChatOpenAI
!pip install -U langchain-community
! pip install chromadb

In [ ]:
%pip install -qU langchain-community beautifulsoup4  # For scraping data we will use beautifulsoup4

In [ ]:
import os
from openai import OpenAI
from google.colab import userdata

# Access the API key from Google Colab secrets
openai_api_key = userdata.get("OPENAI_API_KEY")

# Initialize the OpenAI client
client = OpenAI(api_key=openai_api_key)

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of sitemap
sitemap_url = "https://www.deere.co.in/en/sitemap.xml"

# Fetch sitemap
res = requests.get(sitemap_url)
soup = BeautifulSoup(res.content, "xml")

# Extract all <loc> tags
urls = [loc.text for loc in soup.find_all("loc")]

print(f"Collected {len(urls)} URLs")
print(urls[:10])  # show first 10

Collected 195 URLs
['https://www.deere.co.in/en/finance/financing/', 'https://www.deere.co.in/en/agriculture/', 'https://www.deere.co.in/en/our-company/john-deere-careers/', 'https://www.deere.co.in/en/tractors/', 'https://www.deere.co.in/en/john-deere-technology-center/', 'https://www.deere.co.in/en/parts-service/', 'https://www.deere.co.in/en/parts-service/services-support/', 'https://www.deere.co.in/en/tractors/d-series-tractors/5036d-tractor/', 'https://www.deere.co.in/en/tractors/d-series-tractors/5039d-tractor/', 'https://www.deere.co.in/en/tractors/d-series-tractors/5045d-tractor/']


# Loading The document
We need to pass on all urls, so we are Going to find all urls with India version john deere website and pass onto the loader, use below code to get url details

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader_full_website=WebBaseLoader(urls)
pages_full_website=loader_full_website.load()
print(len(pages_full_website))


195


In [ ]:
print(type(pages_full_website))

<class 'list'>


In [ ]:
print(pages_full_website)

[Document(metadata={'source': 'https://www.deere.co.in/en/finance/financing/', 'title': 'Tractor Financing Solutions for Farmers - John Deere India', 'description': "Invest in farming operations with John Deere India's flexible tractor loan solutions, designed to help grow your business & maximize profitability. Learn more!\n", 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\nTractor Financing Solutions for Farmers - John Deere India\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n   \n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n \n \n\n\n\n Locate a Dealer \n \n\n\n Tractor Price \n \n\n\n\nEnglish\n\n English \n Hindi \n Telugu \n Tamil \n Bengali \n Kannada \n \n  \n\n\n\n\n\n\nBack\n\n Home \n Products \n Finance \n Parts & Service \n \n\n\n\n Locate a Dealer \n \n\n\n Tractor Price \n\n\n Agriculture \n Tractors \n Precision Ag \n Tractor Implements \n Production Systems \n Blogs \n View All Equipment \n\n\n Financing \n\n

In [ ]:
print(pages_full_website[1].page_content[0:])












Agriculture Equipment and Solutions | John Deere India

































   














Skip to main content















Search



 
 



 Locate a Dealer 
 


 Tractor Price 
 



English

 English 
 Hindi 
 Telugu 
 Tamil 
 Bengali 
 Kannada 
 
  






Back

 Home 
 Products 
 Finance 
 Parts & Service 
 



 Locate a Dealer 
 


 Tractor Price 


 Agriculture 
 Tractors 
 Precision Ag 
 Tractor Implements 
 Production Systems 
 Blogs 
 View All Equipment 


 Financing 


 Parts & Service 
 Tractor Warranty 



Agriculture



TRACTORS
 D Series Tractors 
 E Series Tractors 
 Speciality Tractors 
 5M Series Tractors 


PRECISION AG
 Display Gen - 4 (4240) 
 Steering ATU 300 
 Receiver StarFire™ 6000 


IMPLEMENTS
 View Our Implements 





Tractors



D SERIES TRACTORS
 5036D (36 HP, 2WD) 
 5105 (40 HP, 2WD & 4WD) 
 5039D PowerPro™ (41 HP, 2WD) 
 5042D PowerPro™ (44 HP, 2WD) 
 5045D PowerPro™ (46 HP, 2WD & 4WD) 
 5045D GearPro™ (46 HP, 2WD & 4WD) 
 520

# Arranging the document to remove White spaces, taken the code from ChatGPT, made it more organised, validated the output, befor chunking but first lets relook at the structure of scrapped data in details

In [ ]:
# Basic char counts for every page
# Out of curiosity, took the code from chatgPT to get the character counts
char_counts = [len(d.page_content or "") for d in pages_full_website]
print(char_counts)

print("Total pages:", len(pages_full_website))
print("Total chars:", sum(char_counts))
print("Avg chars/page:", round(sum(char_counts) / max(1, len(char_counts)), 2))
print("Min/Max chars:", min(char_counts, default=0), "/", max(char_counts, default=0))

# Top 5 longest pages (by characters)
top_idx = sorted(range(len(char_counts)), key=lambda i: char_counts[i], reverse=True)[:5]
for i in top_idx:
    print(f"[{i}] {char_counts[i]} chars | source = {pages_full_website[i].metadata.get('source')}")


[5596, 3707, 3517, 14883, 19252, 4009, 6169, 9637, 14353, 20410, 21936, 4045, 6303, 7058, 6318, 6834, 8519, 4843, 2876, 6494, 5020, 6951, 5005, 2593, 2526, 2521, 6789, 7272, 6633, 8355, 226, 13623, 13114, 11552, 9628, 9568, 9506, 6379, 7316, 6761, 4698, 4670, 9402, 6324, 5152, 11099, 5585, 6361, 2563, 23567, 6418, 6740, 6460, 6509, 6844, 6690, 7126, 6322, 9602, 226, 228, 6558, 6504, 5814, 10353, 6953, 4911, 2412, 18338, 3188, 3599, 4176, 4553, 5047, 7831, 2728, 5152, 6494, 6494, 2574, 4720, 30124, 12947, 24568, 4629, 27, 226, 30124, 516, 3661, 5076, 1673, 1799, 3441, 112, 3609, 4073, 39, 593, 603, 589, 7186, 568, 564, 150, 6494, 7854, 5502, 6675, 8136, 7156, 8347, 8347, 8347, 8347, 8347, 8347, 2624, 2557, 10919, 4515, 6383, 7522, 5844, 6573, 2624, 2624, 5548, 13285, 2984, 5026, 6823, 4092, 2624, 3799, 5159, 4868, 8417, 15763, 7118, 5947, 6749, 6942, 10203, 22870, 7306, 10944, 10394, 10036, 6369, 10620, 9493, 5958, 8105, 9869, 12082, 9226, 8740, 6584, 10632, 3441, 5342, 6644, 6086, 5, 2

In [ ]:
print(pages_full_website[0].page_content) # we see lots of white spaces here, so its better to normalize the data first












Tractor Financing Solutions for Farmers - John Deere India






























   









Skip to main content















Search



 
 



 Locate a Dealer 
 


 Tractor Price 
 



English

 English 
 Hindi 
 Telugu 
 Tamil 
 Bengali 
 Kannada 
 
  






Back

 Home 
 Products 
 Finance 
 Parts & Service 
 



 Locate a Dealer 
 


 Tractor Price 


 Agriculture 
 Tractors 
 Precision Ag 
 Tractor Implements 
 Production Systems 
 Blogs 
 View All Equipment 


 Financing 


 Parts & Service 
 Tractor Warranty 



Agriculture



TRACTORS
 D Series Tractors 
 E Series Tractors 
 Speciality Tractors 
 5M Series Tractors 


PRECISION AG
 Display Gen - 4 (4240) 
 Steering ATU 300 
 Receiver StarFire™ 6000 


IMPLEMENTS
 View Our Implements 





Tractors



D SERIES TRACTORS
 5036D (36 HP, 2WD) 
 5105 (40 HP, 2WD & 4WD) 
 5039D PowerPro™ (41 HP, 2WD) 
 5042D PowerPro™ (44 HP, 2WD) 
 5045D PowerPro™ (46 HP, 2WD & 4WD) 
 5045D GearPro™ (46 HP, 2WD & 4WD) 
 5205 (4

In [ ]:
# I used chatgpt help to get this normalisation code
# 1) normalize whitespace (recommended)
import re, html
from langchain.schema import Document

def normalize_doc(d: Document) -> Document:
    t = d.page_content or ""
    t = html.unescape(t)
    t = t.replace("\xa0", " ")
    t = re.sub(r"[\u200B-\u200D\uFEFF]", "", t)  # zero-width
    t = re.sub(r"[ \t]+", " ", t)                # collapse spaces/tabs
    t = re.sub(r"\n{3,}", "\n\n", t)             # limit blank lines
    t = re.sub(r" *\n *", "\n", t).strip()       # trim around newlines
    return Document(page_content=t, metadata=d.metadata)

clean_docs = [normalize_doc(d) for d in pages_full_website]
print(clean_docs[0].page_content)




Tractor Financing Solutions for Farmers - John Deere India



Skip to main content

Search




Locate a Dealer


Tractor Price


English

English
Hindi
Telugu
Tamil
Bengali
Kannada



Back

Home
Products
Finance
Parts & Service


Locate a Dealer


Tractor Price

Agriculture
Tractors
Precision Ag
Tractor Implements
Production Systems
Blogs
View All Equipment

Financing

Parts & Service
Tractor Warranty

Agriculture

TRACTORS
D Series Tractors
E Series Tractors
Speciality Tractors
5M Series Tractors

PRECISION AG
Display Gen - 4 (4240)
Steering ATU 300
Receiver StarFire™ 6000

IMPLEMENTS
View Our Implements

Tractors

D SERIES TRACTORS
5036D (36 HP, 2WD)
5105 (40 HP, 2WD & 4WD)
5039D PowerPro™ (41 HP, 2WD)
5042D PowerPro™ (44 HP, 2WD)
5045D PowerPro™ (46 HP, 2WD & 4WD)
5045D GearPro™ (46 HP, 2WD & 4WD)
5205 (48 HP, 2WD)
5050D (50 HP, 2WD & 4WD)
5050D GearPro™ (50 HP, 2WD & 4WD)
5042D GearPro™ PowerPro™ (44HP, 2100 RPM)
5045D PowerPro™ GearPro™ LiftPro™ (46HP, 2100 RPM)

E SERIES TRACTORS

In [ ]:
# 2) chunk with sensible fallbacks
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=150,
    separators=["\n\n", "\n", " ", ""],  # paragraph → line → word → char
)
chunks = splitter.split_documents(clean_docs)
print(chunks)
print(len(chunks))

[Document(metadata={'source': 'https://www.deere.co.in/en/finance/financing/', 'title': 'Tractor Financing Solutions for Farmers - John Deere India', 'description': "Invest in farming operations with John Deere India's flexible tractor loan solutions, designed to help grow your business & maximize profitability. Learn more!\n", 'language': 'en'}, page_content='Tractor Financing Solutions for Farmers - John Deere India\n\n\n\nSkip to main content\n\nSearch\n\n\n\n\nLocate a Dealer\n\n\nTractor Price\n\n\nEnglish\n\nEnglish\nHindi\nTelugu\nTamil\nBengali\nKannada\n\n\n\nBack\n\nHome\nProducts\nFinance\nParts & Service\n\n\nLocate a Dealer\n\n\nTractor Price\n\nAgriculture\nTractors\nPrecision Ag\nTractor Implements\nProduction Systems\nBlogs\nView All Equipment\n\nFinancing\n\nParts & Service\nTractor Warranty\n\nAgriculture\n\nTRACTORS\nD Series Tractors\nE Series Tractors\nSpeciality Tractors\n5M Series Tractors\n\nPRECISION AG\nDisplay Gen - 4 (4240)\nSteering ATU 300\nReceiver StarFi

# Embeddings

In [ ]:
import os
import openai
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=userdata.get("OPENAI_API_KEY"),  # pass explicitly
)
vec = embedding.embed_query("Hello embeddings 👋")

print(len(vec))

/tmp/ipython-input-1090692094.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(


1536


# Vector Stores

In [ ]:

from langchain.vectorstores import Chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.4 MB/s eta

In [ ]:
persist_directory = 'docs/chroma/'

In [ ]:
!rm -rf ./docs/chroma  # remove old database files if any

# Now In order to vaoid duplicacy added this lIne of code to avoid duplicacy of same embeddings Time and Again..

In [ ]:
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,

)

In [ ]:
print(vectordb._collection.count())

1686


Now test the data stored in vectordb by asking questions

Similarity Search

In [ ]:
# Lets Verify, how the data is retrieved from vector store
question="What financing options are available through John Deere Financial in India?"


In [ ]:
retrieve_docs = vectordb.similarity_search(question,k=3) # Langchain wrappers auto embed the query with this command

In [ ]:
len(retrieve_docs)

3

In [ ]:
retrieve_docs[0].page_content

'How can I apply for John Deere tractor finance in India?\n\nWe offer customized solutions that help farmers achieve their desired level of efficiency. When lending to farmers, John Deere Financial considers the farmer’s income sources and financial strength, alongside the production value of the farm as a whole and may finance up to 90% on equipment. To avail a John Deere Tractor finance in India, you may visit your nearest dealership or visit our website on the link below: https://www.deere.co.in/en/request-a-call-back/tractor-pricelist/\n\nWhat is the repayment period for John Deere Finance loans?\n\nWe offer monthly or quarterly and semi-annual installments based on a farmer’s cropping pattern and cash flow to best suit their need.\n\nCan I get a loan for John Deere implements through John Deere Finance?'

Now in case we want to know the source , then we can also check the source through metadata and loop through retrieved documents

In [ ]:
for doc in retrieve_docs:
    print(doc.metadata)

{'language': 'en', 'description': "Buy John Deere India's brand new farm tractor and experience high performance and efficiency in agriculture. Check our models, price, & specifications today.", 'title': 'Buy Farm Tractor | Price & Specifications | John Deere IN', 'source': 'https://www.deere.co.in/en/faq/'}
{'description': 'Discover expert tips on securing tractor financing in India. Empower your farming journey with the right financial support tailored to your needs.', 'language': 'en', 'source': 'https://www.deere.co.in/en/blogs/empowering-indian-farmers/', 'title': 'Tractor Financing Tips: Empowering Indian Farmers | John Deere IN'}
{'description': 'Discover expert tips on securing tractor financing in India. Empower your farming journey with the right financial support tailored to your needs.', 'language': 'en', 'source': 'https://www.deere.co.in/en/blogs/empowering-indian-farmers/', 'title': 'Tractor Financing Tips: Empowering Indian Farmers | John Deere IN'}


In [ ]:
vectordb.persist()

/tmp/ipython-input-3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
# Now lets ask an irrelevant question as per the documents saved and check the output
irrelevant_question = "How do we make Biryani"

In [ ]:
irrelevant_retrieve_docs = vectordb.similarity_search(irrelevant_question,k=3)

In [ ]:
irrelevant_retrieve_docs[0].page_content

'Implement PriceImplement Loan\n\nView Product Brochure\n\nShare\nSHARE\n\nPrint\n\nGreenSystem Square Baler & Rotary Rake\n\nImplement Price\n\nOverview\n\nLook Out For :\n\nThe new Square baler and rotary rake implement Combination is here to ensure every need of straw Management\nThe rotary rake efficiently collects straw after harvesting and the Square baler makes perfectly sized bales which are easier for the labor collect and transport. The efficient Straw collection by rotary rake and picking system of square baler, customer can experience increase in productivity\n\nFeatures\n\nFeatures\n\nExpand All\nCollapse All\n\nFeatures & Benefits: Square Baler (SB1179)\n\n1. Bale density settings\n\nBenefits - Versatility of getting same baler used for different crops (rice & sugarcane)\n2. Balanced flywheel and Powerful gearbox\n\nBenefits - Reduced vibrations provide stability in operations and leads to more life of machine\n3. Strong & Durable Transmission'

# On testing found That , our similarity search is still reverting an output, which is not at all related, so ideated with ChatGPT for a while and developed this threshold scoring concept like 80% threshold, only then share the details or else say i dont know

In [ ]:
docs_scores = vectordb.similarity_search_with_score("How do we make Biryani", k=5)

THRESH = 0.60  # similarity threshold
good = []

for d, dist in docs_scores:
    # Convert cosine distance [0,2] -> similarity [1..-1]
    sim = 1 - dist/2
    if sim >= THRESH:
        good.append((d, sim))

if not good:
    print("No relevant context found. (Answer: I don't have info on that.)")
else:
    for d, s in good:
        print(f"Similarity={s:.2f} | Source={d.metadata} | {d.page_content[:200]}")


No relevant context found. (Answer: I don't have info on that.)


In [ ]:
docs_scores = vectordb.similarity_search_with_score("What financing options are available through John Deere Financial in India?", k=5)


THRESH = 0.60  # similarity threshold
good = []

for d, dist in docs_scores:
    # Convert cosine distance [0,2] -> similarity [1..-1]
    sim = 1 - dist/2
    if sim >= THRESH:
        good.append((d, sim))

if not good:
    print("No relevant context found. (Answer: I don't have info on that.)")
else:
    for d, s in good:
        print(f"Similarity={s:.2f} | Source={d.metadata} | {d.page_content[:200]}")



Similarity=0.83 | Source={'title': 'Buy Farm Tractor | Price & Specifications | John Deere IN', 'description': "Buy John Deere India's brand new farm tractor and experience high performance and efficiency in agriculture. Check our models, price, & specifications today.", 'language': 'en', 'source': 'https://www.deere.co.in/en/faq/'} | How can I apply for John Deere tractor finance in India?

We offer customized solutions that help farmers achieve their desired level of efficiency. When lending to farmers, John Deere Financial consi
Similarity=0.82 | Source={'description': 'Discover expert tips on securing tractor financing in India. Empower your farming journey with the right financial support tailored to your needs.', 'language': 'en', 'title': 'Tractor Financing Tips: Empowering Indian Farmers | John Deere IN', 'source': 'https://www.deere.co.in/en/blogs/empowering-indian-farmers/'} | How does John Deere Financial support farmers' cash flow?
By offering monthly, quarterly, or semi-an

# MMR Stands For maximum Marginal relevance
#How to enforce diversity in the search results.

Maximum marginal relevance strives to achieve both relevance to the query and diversity among the results

In [ ]:
# Lets Verify, how the data is retrieved from vector store
question="What financing options are available through John Deere Financial in India?"


In [ ]:
retrieve_docs = vectordb.similarity_search(question,k=5)   # Basic Similarity Search


In [ ]:
retrieve_docs[0].page_content

'How can I apply for John Deere tractor finance in India?\n\nWe offer customized solutions that help farmers achieve their desired level of efficiency. When lending to farmers, John Deere Financial considers the farmer’s income sources and financial strength, alongside the production value of the farm as a whole and may finance up to 90% on equipment. To avail a John Deere Tractor finance in India, you may visit your nearest dealership or visit our website on the link below: https://www.deere.co.in/en/request-a-call-back/tractor-pricelist/\n\nWhat is the repayment period for John Deere Finance loans?\n\nWe offer monthly or quarterly and semi-annual installments based on a farmer’s cropping pattern and cash flow to best suit their need.\n\nCan I get a loan for John Deere implements through John Deere Finance?'

In [ ]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=5)

In [ ]:
docs_mmr[0].page_content

'How can I apply for John Deere tractor finance in India?\n\nWe offer customized solutions that help farmers achieve their desired level of efficiency. When lending to farmers, John Deere Financial considers the farmer’s income sources and financial strength, alongside the production value of the farm as a whole and may finance up to 90% on equipment. To avail a John Deere Tractor finance in India, you may visit your nearest dealership or visit our website on the link below: https://www.deere.co.in/en/request-a-call-back/tractor-pricelist/\n\nWhat is the repayment period for John Deere Finance loans?\n\nWe offer monthly or quarterly and semi-annual installments based on a farmer’s cropping pattern and cash flow to best suit their need.\n\nCan I get a loan for John Deere implements through John Deere Finance?'

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text.

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this.

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [ ]:


from langchain_openai import ChatOpenAI
from langchain.retrievers.document_compressors import LLMChainExtractor

# Initialize LangChain LLM wrapper (ChatOpenAI, not OpenAI)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=userdata.get("OPENAI_API_KEY")  # or from env var
)

# Now wrap with compressor
compressor = LLMChainExtractor.from_llm(llm)


In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [ ]:
# Lets Verify, how the data is retrieved from vector store
question="What financing options are available through John Deere Financial in India?"

In [ ]:
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

/tmp/ipython-input-773383336.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  compressed_docs = compression_retriever.get_relevant_documents(question)


Document 1:

We offer customized solutions that help farmers achieve their desired level of efficiency. When lending to farmers, John Deere Financial considers the farmer’s income sources and financial strength, alongside the production value of the farm as a whole and may finance up to 90% on equipment. To avail a John Deere Tractor finance in India, you may visit your nearest dealership or visit our website on the link below: https://www.deere.co.in/en/request-a-call-back/tractor-pricelist/

We offer monthly or quarterly and semi-annual installments based on a farmer’s cropping pattern and cash flow to best suit their need.
----------------------------------------------------------------------------------------------------
Document 2:

John Deere India provides financing options for tractors, harvester, implements, and even pre-owned equipment, catering to a wide range of farming needs.
--------------------------------------------------------------------------------------------------

Other types of retrieval¶
It's worth noting that vectordb as not the only kind of tool to retrieve documents.

The LangChain retriever abstraction includes other ways to retrieve documents, such as TF-IDF or SVM.

In [ ]:
# Initialize LangChain LLM wrapper (ChatOpenAI, not OpenAI)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=userdata.get("OPENAI_API_KEY")  # or from env var
)


In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
result = qa_chain({"query": question})

/tmp/ipython-input-4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [ ]:
result["result"]

"John Deere Financial in India offers financing options for tractors, harvesters, implements, and even pre-owned equipment. They provide flexible lending periods, convenient repayment structures, and can finance up to 90% on equipment. Additionally, they offer customized solutions that align with farmers' cash flow, allowing for monthly, quarterly, or semi-annual installments based on cropping patterns."

In [ ]:
question = "What are the top selling tractors?"
result = qa_chain({"query": question })

In [ ]:
result["result"]

'The top selling tractors from John Deere in India include:\n\n1. John Deere 5036D (36 HP, 2WD)\n2. John Deere 5105 (40 HP, 2WD & 4WD)\n3. John Deere 5039D PowerPro™ (41 HP, 2WD)\n4. John Deere 5042D PowerPro™ (44 HP, 2WD)\n5. John Deere 5045D PowerPro™ (46 HP, 2WD & 4WD)\n6. John Deere 5045D GearPro™ (46 HP, 2WD & 4WD)\n7. John Deere 5205 (48 HP, 2WD)\n8. John Deere 5050D (50 HP, 2WD & 4WD)\n9. John Deere 5050D GearPro™ (50 HP, 2WD & 4WD)\n\nThese tractors are known for their performance, fuel efficiency, and versatility in various agricultural operations.'

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)